In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('ev_charging_patterns.csv')
data.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


In [ ]:
# target variable: Efficiency of charge
data['Charging Efficiency (kWh/h)'] = data['Energy Consumed (kWh)'] / data['Charging Duration (hours)']

data_cleaned = data.drop(columns=['User ID', 'Charging Station ID', 'Charging Start Time', 'Charging End Time',
                                        'Charging Station Location', 'Time of Day', 'User Type', 'Day of Week'])
ev_data_cleaned = data_cleaned.dropna()
ev_data_cleaned.head()


,Vehicle Model,Battery Capacity (kWh),Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,Charging Efficiency (kWh/h)
0,BMW i3,108.463007,60.712346,0.591363,36.389181,13.087717,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,102.665033
1,Hyundai Kona,100.000000,12.339275,3.133652,30.677735,21.128448,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,3.937666
2,Chevy Bolt,75.000000,19.128876,2.452653,27.513593,35.667270,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,7.799260
3,Hyundai Kona,50.000000,79.457824,1.266431,32.882870,13.036239,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,62.741544
4,Hyundai Kona,50.000000,19.629104,2.019765,10.215712,10.161471,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,9.718509


In [ ]:
ev_data_preprocessed = pd.get_dummies(ev_data_cleaned, columns=['Vehicle Model', 'Charger Type'])

X = ev_data_preprocessed.drop(columns=['Charging Efficiency (kWh/h)'])
y = ev_data_preprocessed['Charging Efficiency (kWh/h)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((904, 18), (227, 18), (904,), (227,))

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def objective_function(vector):
    """returns the mean squared error of the RandomForest model using the given hyperparameters"""
    n_estimators = int(vector[0])
    max_depth = int(vector[1])

    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

def random_vector(minmax):
    """generate a bounded random approximation to the solution"""
    return minmax[:,0] + (minmax[:,1] - minmax[:,0]) * np.random.random(len(minmax))

def create_particle(search_space, vel_space):
    """create a particle in a random position"""
    particle = {}
    particle['position'] = random_vector(search_space)
    particle['cost'] = objective_function(particle['position'])
    particle['b_position'] = np.copy(particle['position'])
    particle['b_cost'] = particle['cost']
    particle['velocity'] = random_vector(vel_space)
    return particle

def get_global_best(population, current_best = None):
    """get the best global particle"""
    population.sort(key = lambda p: p['cost'])
    best = population[0]
    if current_best == None or best['cost'] <= current_best['cost']:
        current_best = {}
        current_best['position'] = np.copy(best['position'])
        current_best['cost'] = best['cost']
    return current_best

def update_velocity(particle, gbest, max_v, c1, c2):
    for i,v in enumerate(particle['velocity']):
        v1 = c1 * np.random.random() * (particle['b_position'][i] - particle['position'][i])
        v2 = c2 * np.random.random() * (gbest['position'][i] - particle['position'][i])
        particle['velocity'][i] = v + v1 + v2
        if particle['velocity'][i] > max_v: particle['velocity'][i] = max_v
        if particle['velocity'][i] < -max_v: particle['velocity'][i] = -max_v

def update_position(part, bounds):
    """update the position of a particle"""
    for i,v in enumerate(part['position']):
        part['position'][i] = v + part['velocity'][i]
        if part['position'][i] > bounds[i][1]:
            part['position'][i] = bounds[i][1] - abs(part['position'][i]-bounds[i][1])
            part['velocity'][i] *= -1.0
        elif part['position'][i] < bounds[i][0]:
            part['position'][i] = bounds[i][0] + abs(part['position'][i]-bounds[i][0])
            part['velocity'][i] *= -1.0

def update_best_position(particle):
    """update the best position of a particle"""
    if particle['cost'] <= particle['b_cost']:
        particle['b_cost'] = particle['cost']
        particle['b_position'] = np.copy(particle['position'])

# Main PSO function with adjusted parameters
def pso(max_gens, search_space, vel_space, pop_size, max_vel, c1, c2):
    """implements the Particle Swarm Optimization Algorithm for hyperparameter tuning"""
    pop = [create_particle(search_space, vel_space) for i in range(pop_size)]
    gbest = get_global_best(pop)

    for gen in range(max_gens):
        for i,particle in enumerate(pop):
            update_velocity(particle, gbest, max_vel, c1, c2)
            update_position(particle, search_space)
            particle['cost'] = objective_function(particle['position'])
            update_best_position(particle)
            pop[i] = particle
        gbest = get_global_best(pop, gbest)
        print(f"Generation {gen+1} | Best MSE: {gbest['cost']} | Best Position: {gbest['position']}")

    return gbest

# Parameters for RandomForest
problem_size = 2
search_space = np.array([[10, 200], [1, 30]], float)
vel_space = np.array([[-10, 10], [-5, 5]], float)
max_gens = 5
pop_size = 5
max_vel = 10.0
c1, c2 = 2.0, 2.0

# Run PSO algorithm
best_solution = pso(max_gens, search_space, vel_space, pop_size, max_vel, c1, c2)
best_solution


Generation 1 | Best MSE: 55.106708979376606 | Best Position: [134.21185223  11.53410476]
Generation 2 | Best MSE: 55.106708979376606 | Best Position: [134.21185223  11.53410476]
Generation 3 | Best MSE: 55.106708979376606 | Best Position: [134.21185223  11.53410476]
Generation 4 | Best MSE: 55.106708979376606 | Best Position: [134.21185223  11.53410476]
Generation 5 | Best MSE: 55.106708979376606 | Best Position: [134.21185223  11.53410476]


{'position': array([134.21185223,  11.53410476]), 'cost': 55.106708979376606}

In [ ]:
best_n_estimators = int(best_solution['position'][0])
best_max_depth = int(best_solution['position'][1])

final_model = RandomForestRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth, random_state=42)
final_model.fit(X_train, y_train)

y_pred_final = final_model.predict(X_test)
final_mse = mean_squared_error(y_test, y_pred_final)

# Print the results
print(f"Best Hyperparameters found by PSO: n_estimators={best_n_estimators}, max_depth={best_max_depth}")
print(f"Final Model MSE: {final_mse}")

results_df = pd.DataFrame({
    'Actual Charging Efficiency': y_test,
    'Predicted Charging Efficiency': y_pred_final
})
results_df.head()


Best Hyperparameters found by PSO: n_estimators=134, max_depth=11
Final Model MSE: 55.106708979376606


,Actual Charging Efficiency,Predicted Charging Efficiency
914,214.330812,107.548365
1044,13.807431,13.568833
478,4.734573,5.355879
544,84.870011,80.692939
867,27.926351,27.046000


In [ ]:
from openai import OpenAI

def generar_informe_eficiencia(resultados_modelo, detalles_datos, metricas_clave):
    """
    Genera un análisis detallado sobre la eficiencia de carga de vehículos eléctricos utilizando la API de OpenAI.

    Args:
        resultados_modelo: string con los principales resultados de la predicción del modelo.
        detalles_datos: información relevante sobre el dataset utilizado.
        metricas_clave: métrica(s) clave como MSE u otras estadísticas del modelo.

    Returns:
        str: Informe generado por el modelo
    """
    client = OpenAI(
        api_key= 'OPEN AI API KEY'
    )

    prompt = f"""
    Genera un informe detallado sobre los resultados del modelo de predicción de eficiencia de carga de vehículos eléctricos.
    A continuación se presentan los principales resultados del modelo: {resultados_modelo}.
    También se proporciona información sobre los datos utilizados: {detalles_datos}.
    Las métricas clave del modelo son: {metricas_clave}.

    El informe debe explicar de manera clara los siguientes puntos:
    1. El desempeño del modelo en términos de la precisión de predicción (como MSE).
    2. Explicación de los patrones de carga de los vehículos eléctricos según los datos.
    3. Recomendaciones para mejorar la eficiencia de carga basadas en los resultados.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Eres un analista de datos especializado en eficiencia energética y vehículos eléctricos."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0.7
        )

        content = response.choices[0].message.content.strip()

        print("Respuesta de OpenAI (raw):", content)

        return content

    except Exception as e:
        print(f"Error generando el informe de eficiencia de carga: {str(e)}")
        return None

if __name__ == "__main__":
    resultados = "El modelo tiene un error cuadrático medio (MSE) de 53.9. Los valores predichos se alinean bien con los valores reales."
    detalles_datos = "El dataset incluye información de vehículos eléctricos como capacidad de batería, energía consumida, y duración de la carga."
    metricas_clave = "MSE: 53.9, Número de árboles: 150, Profundidad máxima: 10."

    # Generar el informe
    informe_generado = generar_informe_eficiencia(resultados, detalles_datos, metricas_clave)
    if informe_generado:
        print("\nInforme generado:")
        print(informe_generado)

Respuesta de OpenAI (raw): **Informe de Análisis de Eficiencia de Carga de Vehículos Eléctricos**

**1. Desempeño del Modelo:**
El modelo de predicción de eficiencia de carga de vehículos eléctricos ha mostrado un error cuadrático medio (MSE) de 53.9. Esta métrica indica que, en promedio, las predicciones del modelo tienen un error cuadrático de 53.9 unidades en comparación con los valores reales. Aunque el MSE no es muy bajo, se considera aceptable dada la complejidad de los datos y el proceso de carga de vehículos eléctricos. Además, se destaca que los valores predichos se alinean bien con los valores reales, lo que sugiere que el modelo es capaz de capturar la tendencia general de la eficiencia de carga.

**2. Patrones de Carga de Vehículos Eléctricos:**
Según los datos utilizados en el modelo, se observan varios patrones de carga de vehículos eléctricos. Algunos de los factores clave que influyen en la eficiencia de carga son la capacidad de la batería, la energía consumida durante